In [1]:
import pandas as pd
from google.cloud import bigquery
from google.cloud import storage

In [2]:
client = bigquery.Client()

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
query = """select level_3,product_type,attribute,taxonomy_key,data_type,requirement_level,closed_list,multiselect,acceptable_values,
example_values,acceptable_values_es,example_values_es,acceptable_units,definition,date_id 
from wmt-mlp-p-intlctlg-intlctlg.ae_intl_ca.omni_spec_flattened_ca_2 where date_id = '2024-11-27'"""

In [5]:
df = client.query(query).result().to_dataframe()

I0000 00:00:1735014213.084716 1156433 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


In [6]:
pd.set_option('display.max_columns', None)
df[df['product_type']=='Televisions'].head()

,level_3,product_type,attribute,taxonomy_key,data_type,requirement_level,closed_list,multiselect,acceptable_values,example_values,acceptable_values_es,example_values_es,acceptable_units,definition,date_id
36817,Displays,Televisions,Number of Speakers,number_of_speakers,Integer,Required,No,No,None,1;2;3,None,None,None,The number of speakers in an item.,2024-11-27
36818,Displays,Televisions,Height (without stand),height_without_stand,Decimal,Required,No,No,None,"21"";19""",None,None,"in,ft",The height of the product (vertical measuremen...,2024-11-27
36819,Displays,Televisions,Height (with stand),height_with_stand,Decimal,Required,No,No,None,"21"";19""",None,None,"in,ft",The height of the product (vertical measuremen...,2024-11-27
36820,Displays,Televisions,Vertical Viewing Angle,vertical_viewing_angle,Integer,Recommended,No,No,None,100°;178°,None,None,º,The maximum angle at which the display can be ...,2024-11-27
36821,Displays,Televisions,Aspect Ratio,aspect_ratio,String,Required,No,No,None,16:09,None,None,None,The proportional relationship between the disp...,2024-11-27


In [7]:
def create_attribute_prompt_es(row):
    attribute = row['taxonomy_key']
    closed_list = row['closed_list']
    acceptable_values = row['acceptable_values']
    example_values = row['example_values']
    acceptable_values_es = row['acceptable_values_es']
    example_values_es = row['example_values_es']
    acceptable_units = row['acceptable_units']
    multi_select = row['multiselect']
    
    prompt = f'"{attribute}": "Any {attribute} found in the Product User Manual'
    if closed_list == 'Yes':
        if (acceptable_values_es is None or '' or not acceptable_values_es):
            prompt += f' with valid options as {acceptable_values}'
        else:
            prompt += f' with valid options as {acceptable_values_es}'
    else:
        if (example_values_es is None or '' or not example_values_es):
            prompt += f' in the format {example_values}'
        else:
            prompt += f' in the format {example_values_es}'
    
    if (acceptable_units is not None) and (not isinstance(acceptable_units, str)):
        if pd.Series(acceptable_units).notna().any():
            prompt += f'.Acceptable units are {acceptable_units}'
    
    if multi_select == 'Yes':
        prompt += '.It can have multiple comma delimited values'
    else:
        prompt += '.It can have only one value'

    prompt += '"'
    
    return prompt

df['attribute_prompt_es'] = df.apply(create_attribute_prompt_es, axis=1)

In [8]:
def create_attribute_prompt(row):
    attribute = row['taxonomy_key']
    closed_list = row['closed_list']
    acceptable_values = row['acceptable_values']
    example_values = row['example_values']
    acceptable_units = row['acceptable_units']
    multi_select = row['multiselect']
    
    prompt = f'"{attribute}": "Any {attribute} found in the Product User Manual'
    if closed_list == 'Yes':
        prompt += f' with valid options as {acceptable_values}'
    else:
        prompt += f' in the format {example_values}'
    
    if (acceptable_units is not None) and (not isinstance(acceptable_units, str)):
        if pd.Series(acceptable_units).notna().any():
            prompt += f'.Acceptable units are {acceptable_units}'
    
    if multi_select == 'Yes':
        prompt += '.It can have multiple comma delimited values'
    else:
        prompt += '.It can have only one value'

    prompt += '"'
    
    return prompt

df['attribute_prompt_en'] = df.apply(create_attribute_prompt, axis=1)

In [9]:
df = df.rename(columns={'date_id': 'spec_date'})

In [10]:
df.head()

,level_3,product_type,attribute,taxonomy_key,data_type,requirement_level,closed_list,multiselect,acceptable_values,example_values,acceptable_values_es,example_values_es,acceptable_units,definition,spec_date,attribute_prompt_es,attribute_prompt_en
0,Engine Replacement Parts,Automotive Drain Plugs,Fastener Head Type,fastener_head_type,String,Recommended,Yes,No,Button Head;Dome Head;Flat Head;Hex Flange Hea...,None,Cabeza de Botón;Cabeza de Cúpula;Cabeza Plana;...,None,None,"Type of Fastener Head depending on the shape, ...",2024-11-27,"""fastener_head_type"": ""Any fastener_head_type ...","""fastener_head_type"": ""Any fastener_head_type ..."
1,Engine Replacement Parts,Automotive Drain Plugs,Size,size,String,Recommended,No,No,None,0.20 mm; 30 mm; 21 in,None,None,None,Overall dimensions of an item.,2024-11-27,"""size"": ""Any size found in the Product User Ma...","""size"": ""Any size found in the Product User Ma..."
2,Engine Replacement Parts,Automotive Drain Plugs,Automotive Drain Plug Type,automotive_drain_plug_type,String,Required,Yes,No,Radiator Drain Plug;Transmission Drain Plug;Oi...,None,Tapón de Drenaje del Radiador;Tapón de Drenaje...,None,None,Type of automotive drain plug depending on the...,2024-11-27,"""automotive_drain_plug_type"": ""Any automotive_...","""automotive_drain_plug_type"": ""Any automotive_..."
3,Engine Replacement Parts,Automotive Drain Plugs,Vehicle Make,vehicle_make,String,Recommended,No,Yes,None,Ford;BMW;Toyota;KIA,None,None,None,"The manufacturer’s marque, under which the veh...",2024-11-27,"""vehicle_make"": ""Any vehicle_make found in the...","""vehicle_make"": ""Any vehicle_make found in the..."
4,Engine Replacement Parts,Automotive Drain Plugs,Vehicle Type,vehicle_type,String,Required,Yes,Yes,Boat;Bus;Car;Dirt Bike;Go-Kart;Golf Cart;Mini ...,None,Barco;Autobús;Coche;Moto de Cross;Go Kart;Carr...,None,None,Grouping of different kinds of vehicles based ...,2024-11-27,"""vehicle_type"": ""Any vehicle_type found in the...","""vehicle_type"": ""Any vehicle_type found in the..."


In [11]:
df.shape

(568727, 17)

In [12]:
df['attribute_prompt_en'].iloc[0]

'"fastener_head_type": "Any fastener_head_type found in the Product User Manual with valid options as Button Head;Dome Head;Flat Head;Hex Flange Head;Hex Head;Hex Washer Head;Oval Head;Pan Head;Phillips Head;Round Head;Slotted Hex Washer Head;Socket Cap Head;Square Head;Truss Head.It can have only one value"'

In [13]:
df['attribute_prompt_es'].iloc[45]

'"series": "Any series found in the Product User Manual in the format GM Original Equipment Series; Motorcraft OE Replacement; Liland OE Replacement; Slim-Fit Series; CSF Radiator; Valeo OE Replacement; GPD OE Replacement; GenuineXL OE Replacement; Liland Radiator; Power BladeRunner Series.It can have only one value"'

In [14]:
bucket_name = 'gs://wmt-mlp-p-intlctlg-export-bucket'
blob_path = 'AE/GenAI'

In [15]:
PT_attribute_path = f"{bucket_name}/{blob_path}/ae_attribute_spec_ca.parquet"

In [16]:
df.to_parquet(PT_attribute_path)